In [34]:
import pandas as pd
import numpy as np
import statistics
import requests
import json
import operator

## Common Methods

In [35]:
def callApi(method, stock, isAnnual = False ):
    key = "token=" + "pk_ee5af08dfb9a419aaba2cbee77b80165"
    host = "https://cloud.iexapis.com/stable/"
    ticker = "stock/"+ stock
    api_method ="/" + method 
    query = "?"
    add = "&"
    annualLast = "/10"
    annualPeriod = "period=annual"
    quarterLast = "/4"
    quarterPeriod = "period=quarter"
    if isAnnual:
        url = host + ticker + api_method + annualLast + query + annualPeriod + add + key 
    else:
        url = host + ticker + api_method + quarterLast + query + quarterPeriod + add + key 
    try:
        response = requests.request("GET", url)
        balance_sheet = json.loads(response.text)
        return balance_sheet
    except ValueError:
        print("Unknown expeption on" + stock)
        return {}

In [36]:
def addTTMToYearLists(method,lists,ttmList):
    reportDate = "reportDate"
    if method == "earnings":
        reportDate = "EPSReportDate"
    if not (method in ttmList.keys()):
        return lists
    first = lists[method][0]
    ttm = ttmList[method][0]
    if first[reportDate] != ttm[reportDate]: 
        lists[method].insert(0, ttmList[method][0])
    return lists

## Ratio Methods

In [37]:
def peRatio(quote):
    if "peRatio" in quote.keys() and quote["peRatio"] is not None:
        return float(quote["peRatio"])
    return 0.0

In [38]:
def priceToBook(stats):
    if "priceToBook" in stats.keys() and stats["priceToBook"] is not None:
        return float(stats["priceToBook"])
    return 0.0

In [39]:
def debtToEquity(stats):
    if "debtToEquity" in stats.keys() and stats["debtToEquity"] is not None:
        return float(stats["debtToEquity"])
    return 0.0 

In [40]:
def pegRatio(stats):
    if"pegRatio" in stats.keys() and stats["pegRatio"] is not None:
        return float(stats["pegRatio"])
    return 0.0

In [41]:
def marketCap(quote):
    if "marketCap" in quote.keys() and quote["marketCap"] is not None:
        return quote["marketCap"]
    return 0 

In [42]:
# Current Ratio= Current Assets / Current Liabilities
def currentRatio(balance_sheet):
    if "balancesheet" in balance_sheet.keys() and len(balance_sheet["balancesheet"]) > 0:
        bs = balance_sheet["balancesheet"][0]
        currentAssets = bs['currentAssets']
        totalCurrentLiabilities = bs['totalCurrentLiabilities']
        if currentAssets is not None and totalCurrentLiabilities is not None and totalCurrentLiabilities != 0:
            return currentAssets / totalCurrentLiabilities
    return 0                                                                                                   

In [43]:
def checkErningProgression(earnings):
    yearAgoChangePercent = np.array([])
    if "earnings" in earnings.keys() and len(earnings["earnings"]) > 0:
        earningsSet = earnings["earnings"][:-1]
        for earning in earningsSet:
            if "yearAgoChangePercent" in earning.keys() and earning["yearAgoChangePercent"] is not None:
                yearAgoChangePercent = np.append(yearAgoChangePercent, (earning["yearAgoChangePercent"]))
    if len(yearAgoChangePercent) == 0 :
        return False
    isDecending = (yearAgoChangePercent == np.sort(yearAgoChangePercent)[::-1]).all()
    return isDecending

In [44]:
def sharesOutstanding(stats):
    if "sharesOutstanding" in stats.keys() and stats["sharesOutstanding"] is not None:
        return float(stats["sharesOutstanding"])
    return 0.0

In [50]:
def NetCurrentAssetValuePerShare(balanceSheet, stats, price):
    #Net Current Asset Value = Current Assets – (Current Liabilities + Long-Term Liabilities) 
    shares_Outstanding = sharesOutstanding(stats)
    if "balancesheet" in balance_sheet.keys() and len(balance_sheet["balancesheet"]) > 0 and shares_Outstanding !=0:
        latest_balance_sheet = balance_sheet["balancesheet"][0]
        if "currentAssets" in latest_balance_sheet.keys() and "totalLiabilities" in latest_balance_sheet.keys() and "minorityInterest" in latest_balance_sheet.keys():
            if latest_balance_sheet["currentAssets"] is not None and latest_balance_sheet["totalLiabilities"] is not None and latest_balance_sheet["minorityInterest"] is not None: 
                netCurrentAssetValue = (latest_balance_sheet["currentAssets"] - latest_balance_sheet["totalLiabilities"] - latest_balance_sheet["minorityInterest"]) / shares_Outstanding
                return (price/netCurrentAssetValue) * 100
    return 0

In [46]:
# Last Six moths data 
def insideTrade(insiderSummary):
    return sum(item['netTransacted'] for item in insiderSummary)

In [47]:
def ZScore(balance_sheet, incomes, quote):

    X1 = X2 = X3 = X4 = X5 = 0
    
    if "balancesheet" in balance_sheet.keys() and len(balance_sheet["balancesheet"]) > 0 and "income" in income.keys() and len(income["income"]) > 0:
        latest_balance_sheet = balance_sheet["balancesheet"][0]
        latest_income = income["income"][0]
        
        #X1 = Working Capital / Total Assets
        # =(Total Current Assets - Total Current Liabilities) / Total Assets
        if "currentAssets" in latest_balance_sheet.keys() and "totalCurrentLiabilities" in latest_balance_sheet.keys() and "totalAssets" in latest_balance_sheet.keys():
            totalCurrentAssets = latest_balance_sheet["currentAssets"]
            totalLiabilities = latest_balance_sheet["totalCurrentLiabilities"]
            totalAssets = latest_balance_sheet["totalAssets"]
            if totalCurrentAssets is not None and totalLiabilities is not None and totalAssets is not None: 
                X1 = (totalCurrentAssets - totalLiabilities) /  totalAssets
        
        #X2 = Retained Earnings /Total Assets    
        if "retainedEarnings" in latest_balance_sheet.keys() and "totalAssets" in latest_balance_sheet.keys():
            retainedEarnings = latest_balance_sheet["retainedEarnings"]
            totalAssets = latest_balance_sheet["totalAssets"]
            if retainedEarnings is not None and totalAssets is not None:
                X2 = retainedEarnings /  totalAssets
        
        #X3 = Earnings Before Interest and Taxes / Total Assets
        #    = (Pre-Tax Income - Interest Expense) / Total Assets
        # Not able to get Interest Expense
        if "pretaxIncome" in latest_income.keys() and "totalAssets" in latest_balance_sheet.keys():
            pretaxIncome = sum(income['pretaxIncome'] is not None for income in incomes["income"])
            totalAssets = latest_balance_sheet["totalAssets"]
            if pretaxIncome is not None and totalAssets is not None:
                X3 = pretaxIncome / totalAssets
        
        
        #X4 = Market Value Equity / Book Value of Total Liabilities
        if "totalLiabilities" in latest_balance_sheet.keys():
            market_value_equity = marketCap(quote)
            totalLiabilities = latest_balance_sheet['totalLiabilities']
            if market_value_equity is not None and totalLiabilities is not None:
                X4 = market_value_equity/ totalLiabilities
            
        #X5 = Revenue / Total Assets
        if "income" in income.keys() and "totalAssets" in latest_balance_sheet.keys():
            totalRevenue = sum(income['totalRevenue'] is not None for income in incomes["income"])
            totalAssets = latest_balance_sheet["totalAssets"]
            if totalRevenue is not None and totalAssets is not None:
                X5 = totalRevenue / totalAssets

    zScore = 1.2 * X1 + 1.4 * X2 + 3.3 * X3 + 0.6 * X4 + 1.0 * X5
        
        #ZScore= 1.2* X1 + 1.4 * X2 + 3.3 * X3 + 0.6 * X4 + 1.0 * X5
    return zScore

In [48]:
def FScore(income, cashflows):
    score = 0
    if "income" in income.keys():
        netIncome = sum(net['netIncome'] is not None for net in income["income"])
        if netIncome > 0:
            score += 1 
    if "cashflow" in cashflows.keys():
        cashflow = sum(flow['cashFlow']  is not None for flow in cashflows["cashflow"]) 
        if cashflow > 0:
            score += 1 
    
    return score 

In [51]:
#tickers = ['AAPL']
tickers = ['CRON','CRWS','JCS','GRBK','ISNS', 'JRSH','LAKE','OPNT','NWPX','RCKY','SMMT','XBIT','TAYD','VNDA', 'WEYS','STRA']
dataSet = [] 
for ticker in tickers:
    data = {}
    points = 0
    
    quote = callApi("quote", ticker)
    
    # Less than 10 million don't comsider
    market_cap =  marketCap(quote)
    if market_cap < 10000000:
        continue
        
    # Z-Score 
    balance_sheet = callApi("balance-sheet", ticker)
    income = callApi("income", ticker)
    zScore = ZScore(balance_sheet, income, quote)
    if zScore is not None and zScore > 0:
        if zScore > 7.99:
            points += 2
        elif zScore > 2.99:
            points += 1
    else:
        continue
    
    # Major data set
    company = callApi("company", ticker)
    stats = callApi("advanced-stats", ticker)
    earnings = callApi("earnings", ticker, True)
    price = callApi("price", ticker)
    inside_trade = callApi("insider-summary", ticker)
    
    
    data["Ticker"] = ticker
    data ["company Name"] = quote["companyName"]
    data["Sector"] = company["sector"]
    data["Price"] = price
    data["Market Cap"] = round(market_cap/1000000,2)  
    data["52 High"] = quote["week52High"]
    data["52 Low"] = quote["week52Low"]
    data["Z Score"] =  round(zScore,2)
    
        
    # Net Current Asset Value Per Share
    # Graham instituted a rule in which he would only consider buying a stock if its current price was trading at lower than 66% of its NCAVPS.
    NCAVPS = NetCurrentAssetValuePerShare(balance_sheet, stats, price)
    data["NCAVPS <50%"] =  round(NCAVPS, 2)
    if NCAVPS < .65 and NCAVPS > 0:
        points += 2
    elif NCAVPS < .50 and NCAVPS > 0:
        points += 1
        
    # PE Ratio < 7
    pe_ratio = peRatio(quote)
    if pe_ratio != 0: 
        data["PE Ratio < 7"] = round(pe_ratio,2)
    else: 
        data["PE Ratio < 7"] = "NA"
    if pe_ratio < 7 and pe_ratio > 0: 
        points += 1
        
    # Price To Book Ratio  
    price_to_book = priceToBook(stats)
    if price_to_book != 0: 
        data["Price To Book < 1"] = round(price_to_book,2)
    else: 
        data["Price To Book < 1"] = "NA"
    
    if  price_to_book < 1 and  price_to_book > 0: 
        points += 1
    
    
    # Debt To Equity Ratio 
    # Warent buffet say < 0.5
    debt_Equity = debtToEquity(stats)
    if debt_Equity != 0: 
        data["DE < 0.5"] = round(debt_Equity,2)
    else: 
        data["DE < 0.5"] = "NA"
    if debt_Equity < 0.5 and debt_Equity > 0:
        points += 1
        
    # Current Ratio 
    current_ratio = currentRatio(balance_sheet)
    if current_ratio != 0: 
        data["CR > 3"] = round(current_ratio,2)
    else: 
        data["CR > 3"] = "NA"
        
    if current_ratio > 3:
        points += 1
    
    # Earning Progression Last 4 year 
    isProgressing = checkErningProgression(earnings)
    data["Earning Progression"] =  isProgressing
    if isProgressing:
        points += 1

        
    # Inside Traide
    trade = insideTrade(inside_trade)
    data["Inside Trade"] =  trade
    if trade > 10000:
        points += 1
        
    data["Points(10)"] = points

    dataSet.append(data.copy())
dataSet.sort(key=operator.itemgetter("Points(10)"), reverse=True)
df = pd.DataFrame(dataSet)  
df.to_csv("Advance-reserch.csv", sep='\t', encoding='utf-8')
pd.DataFrame(dataSet) 

,Ticker,company Name,Sector,Price,Market Cap,52 High,52 Low,Z Score,NCAVPS <50%,PE Ratio < 7,Price To Book < 1,DE < 0.5,CR > 3,Earning Progression,Inside Trade,Points(10)
0,JCS,"Communications Systems, Inc.",Electronic Technology,4.340,40.61,9.90,3.50,3.15,121.27,13.27,0.84,0.01,4.44,True,177698,6
1,OPNT,"Opiant Pharmaceuticals, Inc.",Health Technology,8.860,37.73,18.23,7.35,3.13,112.48,3.61,1.09,0.02,7.58,True,80230,6
2,XBIT,"XBiotech, Inc.",Health Technology,21.120,612.25,26.40,8.18,25.37,254.08,1.30,0.76,NA,18.12,False,46859,6
3,SMMT,Summit Therapeutics Plc,Health Technology,3.350,225.23,5.49,1.18,5.16,434.50,5.57,2.78,0.01,4.80,True,0,5
4,TAYD,"Taylor Devices, Inc.",Producer Manufacturing,9.000,31.39,13.39,6.61,5.18,108.65,10.34,0.79,0.04,6.22,False,27885,5
5,VNDA,"Vanda Pharmaceuticals, Inc.",Health Technology,9.655,527.66,17.85,7.12,4.39,172.54,4.53,1.28,0.04,6.18,False,140631,5
6,WEYS,"Weyco Group, Inc.",Distribution Services,17.150,168.30,26.72,15.40,2.84,267.32,22.29,0.80,0.13,5.84,True,16813,5
7,CRON,"Cronos Group, Inc.",Process Industries,5.510,1927.87,11.77,4.00,6.22,166.87,2.98,1.08,NA,5.96,False,2637367,4
8,GRBK,"Green Brick Partners, Inc.",Consumer Durables,18.575,941.04,18.73,5.66,3.20,187.49,11.56,1.72,0.46,9.11,False,52997,4
9,ISNS,"Image Sensing Systems, Inc.",Electronic Technology,3.800,20.28,6.00,2.79,6.27,234.69,3.28,1.13,NA,7.46,False,16900,4
